### Problema

**"The main goal is to forecast the number of people that uses the metro. You can let the imagination work and do this the way you prefer, chosing the temporal/spatial sampling rate (by station/turnstile, day/hour/week etc). This choice is part of the challenge."**


In [1]:
# data processing:
import pandas as pd
from datetime import datetime
import holidays

import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import scipy.stats as sct

from pylab import rcParams
rcParams['figure.figsize'] = 12, 6

import warnings
warnings.filterwarnings("ignore")


**Importing Data**

In [2]:
df_2011 = pd.read_csv("dados_desafio/2011.csv.gz",compression='gzip')

In [3]:
df_2011.head()

,time,ca,unit,scp,station,linename,division,desc,entries,exits
0,2011-01-01 00:00:00,A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,REGULAR,2994127.0,1033610.0
1,2011-01-01 00:00:00,A002,R051,02-00-01,LEXINGTON AVE,456NQR,BMT,REGULAR,2959715.0,627670.0
2,2011-01-01 00:00:00,A002,R051,02-03-00,LEXINGTON AVE,456NQR,BMT,REGULAR,1235350.0,4380835.0
3,2011-01-01 00:00:00,A002,R051,02-03-01,LEXINGTON AVE,456NQR,BMT,REGULAR,2709282.0,4179651.0
4,2011-01-01 00:00:00,A002,R051,02-03-02,LEXINGTON AVE,456NQR,BMT,REGULAR,2488392.0,3405802.0


In [4]:
print(df_2011.shape),
df_2011.nunique()

(11146415, 10)


time        1536502
ca              727
unit            467
scp             222
station         382
linename        107
division          6
desc             13
entries     5355922
exits       4739088
dtype: int64

### Sobre os dados

**Metadados da base de entrada**


C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
  
  
C/A      = Control Area (A002)  
UNIT     = Remote Unit for a station (R051)  
SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)  
STATION  = Represents the station name the device is located at  
LINENAME = Represents all train lines that can be boarded at this station  
           Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains.  
DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND     
DATE     = Represents the date (MM-DD-YY)  
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event  
DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)  
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities.   
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered.   
ENTRIES  = The comulative entry register value for a device  
EXIST    = The cumulative exit register value for a device


**Entendimento das bases e estudos no site do  [Metrô de NY](https://www.nycsubway.org/wiki/Main_Page) levaram as seguintes conclusões:**  
*(Referência: 2011)*  
    1. Número de Linhas de metrô (incluindo as agrupadas): 107
    2. Número de Estações: 382
    3. Existem registros para estações que acessam à duas linhas, gerando um total "Estações-Linha" de 477
    4. O código de controle de área (CA) dividem áreas de determinadas catracas de cada estação, no total são 727
    5. Cada catraca possui um código (SCP) que é distinto apenas na estação em que se encontra.
    6. O número de entradas e saídas acumuladas ("entries" e "exits") por catraca não tem necessariamente início no primeido dia do ano, ou seja, o ciclo de contagem pode recomeçar no meio do ano causando descontinuidade nos valores.

<img src="panorama_geral.PNG" alt="esboco_mapa" />


In [5]:
df_2011.isna().sum()

time            0
ca              0
unit            0
scp             0
station     18484
linename    18484
division    18484
desc            0
entries         0
exits           0
dtype: int64

In [6]:
df_2011[df_2011.station.isnull()].nunique()

time         5330
ca              1
unit            1
scp             6
station         0
linename        0
division        0
desc            8
entries     13435
exits       13747
dtype: int64

*Os nulos estão em um determinado CA, ou seja, é um conjunto de catracas de uma determinada estação que atende à uma única linha.*

In [7]:
df_2011[df_2011.station.isnull()][df_2011.time == '2011-12-01 00:00:00']

,time,ca,unit,scp,station,linename,division,desc,entries,exits
10207892,2011-12-01 00:00:00,R217,R194,00-00-00,NaN,NaN,NaN,REGULAR,10762894.0,1349534.0
10207893,2011-12-01 00:00:00,R217,R194,00-00-01,NaN,NaN,NaN,REGULAR,2053169.0,327599.0
10207894,2011-12-01 00:00:00,R217,R194,00-00-02,NaN,NaN,NaN,REGULAR,5290440.0,1569678.0
10207895,2011-12-01 00:00:00,R217,R194,00-00-03,NaN,NaN,NaN,REGULAR,11368977.0,5963382.0
10207896,2011-12-01 00:00:00,R217,R194,00-06-00,NaN,NaN,NaN,VLT,2037043.0,3189298.0
10207897,2011-12-01 00:00:00,R217,R194,00-06-01,NaN,NaN,NaN,VLT,84453.0,100511.0


In [8]:
df_2011[['linename', 'division']].drop_duplicates().sort_values(by = 'linename').groupby('division', as_index = False).agg({'linename':'count'})

,division,linename
0,BMT,41
1,IND,50
2,IRT,39
3,PTH,1
4,RIT,1
5,SRT,1


In [9]:
# Por enquanto, vamos chamar a estação e a linha desconhecidas com prefixo x, porém,
# vale lembrar que essa estação pode já fazer parte do nosso conjunto de dados.

df_2011.station.fillna('x_station', inplace = True)
df_2011.linename.fillna('x_linename', inplace = True)
df_2011.division.fillna('IND', inplace = True) # Division mais frequênte

In [59]:
df_2011.head()

,time,ca,unit,scp,station,linename,division,desc,entries,exits
0,2011-01-01 00:00:00,A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,REGULAR,2994127.0,1033610.0
1,2011-01-01 00:00:00,A002,R051,02-00-01,LEXINGTON AVE,456NQR,BMT,REGULAR,2959715.0,627670.0
2,2011-01-01 00:00:00,A002,R051,02-03-00,LEXINGTON AVE,456NQR,BMT,REGULAR,1235350.0,4380835.0
3,2011-01-01 00:00:00,A002,R051,02-03-01,LEXINGTON AVE,456NQR,BMT,REGULAR,2709282.0,4179651.0
4,2011-01-01 00:00:00,A002,R051,02-03-02,LEXINGTON AVE,456NQR,BMT,REGULAR,2488392.0,3405802.0


In [10]:
df_2011['date'] = df_2011.time.str.split(" ").str[0]
df_2011['hour'] = df_2011.time.str.split(" ").str[1]

df_2011['year'] = df_2011.time.str.split(" ").str[0].str.split("-").str[0]
df_2011['month'] = df_2011.time.str.split(" ").str[0].str.split("-").str[1]
df_2011['day'] = df_2011.time.str.split(" ").str[0].str.split("-").str[2]


In [11]:
df_2011.head()

,time,ca,unit,scp,station,linename,division,desc,entries,exits,date,hour,year,month,day
0,2011-01-01 00:00:00,A002,R051,02-00-00,LEXINGTON AVE,456NQR,BMT,REGULAR,2994127.0,1033610.0,2011-01-01,00:00:00,2011,01,01
1,2011-01-01 00:00:00,A002,R051,02-00-01,LEXINGTON AVE,456NQR,BMT,REGULAR,2959715.0,627670.0,2011-01-01,00:00:00,2011,01,01
2,2011-01-01 00:00:00,A002,R051,02-03-00,LEXINGTON AVE,456NQR,BMT,REGULAR,1235350.0,4380835.0,2011-01-01,00:00:00,2011,01,01
3,2011-01-01 00:00:00,A002,R051,02-03-01,LEXINGTON AVE,456NQR,BMT,REGULAR,2709282.0,4179651.0,2011-01-01,00:00:00,2011,01,01
4,2011-01-01 00:00:00,A002,R051,02-03-02,LEXINGTON AVE,456NQR,BMT,REGULAR,2488392.0,3405802.0,2011-01-01,00:00:00,2011,01,01


**Visão:  Estação - catraca - dia  - (entrada máxima acumulada naquele dia)**

In [11]:
entries_day_scp = df_2011.groupby(['station', 'ca','scp', 'linename', 'date'], as_index = False).agg({'entries':'max'})

In [12]:
entries_day_scp.head()

,station,ca,scp,linename,date,entries
0,1 AVE,H007,00-00-00,L,2011-01-01,3744872.0
1,1 AVE,H007,00-00-00,L,2011-01-02,3748661.0
2,1 AVE,H007,00-00-00,L,2011-01-03,3752590.0
3,1 AVE,H007,00-00-00,L,2011-01-04,3756493.0
4,1 AVE,H007,00-00-00,L,2011-01-05,3760553.0


In [14]:
# Chave station_linename_ca_scp para facilitar os tratamentos - Equivalentes as catracas distintas
entries_day_scp['station_linename_ca_scp'] = entries_day_scp['linename'] +entries_day_scp['station'] +entries_day_scp['ca'] +entries_day_scp['scp'] 

In [15]:
print("Existem {} catracas distintas.".format(entries_day_scp.station_linename_ca_scp.nunique()))
df_2011[['station','ca','linename', 'scp']].drop_duplicates().shape, entries_day_scp.station_linename_ca_scp.nunique()

Existem 4455 catracas distintas.


((4455, 4), 4455)

In [16]:
# Agrupamento pela chave para o cálculo de entradas diárias por catraca
grouped = entries_day_scp.groupby('station_linename_ca_scp')

In [17]:
entries_day_scp['n_entries'] = (entries_day_scp['entries'] - grouped['entries'].shift(1))

In [26]:
# n_entries é o número de pessoas que passou por aquela catraca naquele dia.
entries_day_scp

,station,ca,scp,linename,date,entries,station_linename_ca_scp,n_entries,n_entries_corrigido
0,1 AVE,H007,00-00-00,L,2011-01-01,3744872.0,L1 AVEH00700-00-00,0.0,0.0
1,1 AVE,H007,00-00-00,L,2011-01-02,3748661.0,L1 AVEH00700-00-00,3789.0,3789.0
2,1 AVE,H007,00-00-00,L,2011-01-03,3752590.0,L1 AVEH00700-00-00,3929.0,3929.0
3,1 AVE,H007,00-00-00,L,2011-01-04,3756493.0,L1 AVEH00700-00-00,3903.0,3903.0
4,1 AVE,H007,00-00-00,L,2011-01-05,3760553.0,L1 AVEH00700-00-00,4060.0,4060.0
...,...,...,...,...,...,...,...,...,...
1583668,x_station,R217,00-06-01,x_linename,2011-12-27,105395.0,x_linenamex_stationR21700-06-01,575.0,575.0
1583669,x_station,R217,00-06-01,x_linename,2011-12-28,106241.0,x_linenamex_stationR21700-06-01,846.0,846.0
1583670,x_station,R217,00-06-01,x_linename,2011-12-29,107202.0,x_linenamex_stationR21700-06-01,961.0,961.0
1583671,x_station,R217,00-06-01,x_linename,2011-12-30,108239.0,x_linenamex_stationR21700-06-01,1037.0,1037.0


In [19]:
# Como são 4455 catracas, portanto são 4455 valores nulos (correspondentes a primeira linha de subtração)
entries_day_scp.n_entries.isna().sum()

4455

In [21]:
# Substituindo valores nulos
#entries_day_scp.loc[entries_day_scp.n_entries.isna(), 'n_entries_corrigido'] =entries_day_scp.loc[entries_day_scp.n_entries < 0, 'entries']

In [25]:
# n_entries é o número de pessoas que passou por aquela catraca naquele dia.
entries_day_scp['n_entries_corrigido'] = entries_day_scp['n_entries']
entries_day_scp['n_entries_corrigido'].fillna(0)

In [27]:
# n_entries é o número de pessoas que passou por aquela catraca naquele dia.
entries_day_scp

,station,ca,scp,linename,date,entries,station_linename_ca_scp,n_entries,n_entries_corrigido
0,1 AVE,H007,00-00-00,L,2011-01-01,3744872.0,L1 AVEH00700-00-00,0.0,0.0
1,1 AVE,H007,00-00-00,L,2011-01-02,3748661.0,L1 AVEH00700-00-00,3789.0,3789.0
2,1 AVE,H007,00-00-00,L,2011-01-03,3752590.0,L1 AVEH00700-00-00,3929.0,3929.0
3,1 AVE,H007,00-00-00,L,2011-01-04,3756493.0,L1 AVEH00700-00-00,3903.0,3903.0
4,1 AVE,H007,00-00-00,L,2011-01-05,3760553.0,L1 AVEH00700-00-00,4060.0,4060.0
...,...,...,...,...,...,...,...,...,...
1583668,x_station,R217,00-06-01,x_linename,2011-12-27,105395.0,x_linenamex_stationR21700-06-01,575.0,575.0
1583669,x_station,R217,00-06-01,x_linename,2011-12-28,106241.0,x_linenamex_stationR21700-06-01,846.0,846.0
1583670,x_station,R217,00-06-01,x_linename,2011-12-29,107202.0,x_linenamex_stationR21700-06-01,961.0,961.0
1583671,x_station,R217,00-06-01,x_linename,2011-12-30,108239.0,x_linenamex_stationR21700-06-01,1037.0,1037.0


In [29]:
# Cada catraca deve conter aproximadamente 365 registros, referente a cada dia do ano
entries_day_scp[entries_day_scp['station_linename_ca_scp'] == 'L1 AVEH00700-00-01'].shape

(365, 9)

In [30]:
# Existe quebra de ciclo de contagem acumulada das catracas, portanto deve existir valores negativos.
# Exemplo:
entries_day_scp[entries_day_scp.n_entries < 0]

,station,ca,scp,linename,date,entries,station_linename_ca_scp,n_entries,n_entries_corrigido
15484,110 ST,R254,01-00-01,6,2011-07-08,2742.0,6110 STR25401-00-01,-7007840.0,-7007840.0
19912,110 ST-CPN,R301,00-03-01,23,2011-08-27,470.0,23110 ST-CPNR30100-03-01,-756030.0,-756030.0
23665,111 ST,R530,00-00-04,7,2011-12-08,2128.0,7111 STR53000-00-04,-9306030.0,-9306030.0
23886,116 ST,N029,01-00-00,BC,2011-07-17,102.0,BC116 STN02901-00-00,-991289.0,-991289.0
24194,116 ST,N029,01-00-01,BC,2011-05-21,669.0,BC116 STN02901-00-01,-62323.0,-62323.0
...,...,...,...,...,...,...,...,...,...
1568183,WOODHAVEN BLVD,J032,01-05-00,JZ,2011-06-04,0.0,JZWOODHAVEN BLVDJ03201-05-00,-43.0,-43.0
1569242,WOODHAVEN BLVD,J032,01-06-01,JZ,2011-04-29,1143.0,JZWOODHAVEN BLVDJ03201-06-01,-528032.0,-528032.0
1569622,WOODHAVEN BLVD,N329,00-00-00,MR,2011-05-14,6207.0,MRWOODHAVEN BLVDN32900-00-00,-17802281.0,-17802281.0
1583401,x_station,R217,00-06-01,x_linename,2011-03-23,1182336.0,x_linenamex_stationR21700-06-01,-7320962.0,-7320962.0


**Se existem varlores negativos, é porque existem catracas que a contagem acumulada é zerada em um determinado dias, devemos tratar esses caras antes de somar a entrada diária das catracas por estação.**

In [31]:
# EXEMPLO:
# STATION: 110 ST
# SCP: 01-00-01
# DATA RECONTAGEM: 2011-07-08

entries_day_scp[(entries_day_scp.station == '110 ST')
                # &(entries_day_scp.linename ==  6)
                & (entries_day_scp.scp == '01-00-01')
                & ((entries_day_scp.date > '2011-07-05')
                   & (entries_day_scp.date < '2011-07-10'))]

,station,ca,scp,linename,date,entries,station_linename_ca_scp,n_entries,n_entries_corrigido
15482,110 ST,R254,01-00-01,6,2011-07-06,7009652.0,6110 STR25401-00-01,985.0,985.0
15483,110 ST,R254,01-00-01,6,2011-07-07,7010582.0,6110 STR25401-00-01,930.0,930.0
15484,110 ST,R254,01-00-01,6,2011-07-08,2742.0,6110 STR25401-00-01,-7007840.0,-7007840.0
15485,110 ST,R254,01-00-01,6,2011-07-09,4479.0,6110 STR25401-00-01,1737.0,1737.0


In [32]:
# Substituindo valores negativos pelos valores de contagem do dia
entries_day_scp.loc[entries_day_scp.n_entries < 0, 'n_entries_corrigido'] =entries_day_scp.loc[entries_day_scp.n_entries < 0, 'entries']

In [33]:
# EXEMPLO:
# STATION: 110 ST
# SCP: 01-00-01
# DATA RECONTAGEM: 2011-07-08

entries_day_scp[(entries_day_scp.station == '110 ST')
                # &(entries_day_scp.linename ==  6)
                & (entries_day_scp.scp == '01-00-01')
                & ((entries_day_scp.date > '2011-07-05')
                   & (entries_day_scp.date < '2011-07-10'))]

,station,ca,scp,linename,date,entries,station_linename_ca_scp,n_entries,n_entries_corrigido
15482,110 ST,R254,01-00-01,6,2011-07-06,7009652.0,6110 STR25401-00-01,985.0,985.0
15483,110 ST,R254,01-00-01,6,2011-07-07,7010582.0,6110 STR25401-00-01,930.0,930.0
15484,110 ST,R254,01-00-01,6,2011-07-08,2742.0,6110 STR25401-00-01,-7007840.0,2742.0
15485,110 ST,R254,01-00-01,6,2011-07-09,4479.0,6110 STR25401-00-01,1737.0,1737.0


**Valores de entradas em cada "estação-linha" por dia.**

In [55]:
entries_day = entries_day_scp.groupby(['station', 'linename', 'date'], as_index = False).agg({'n_entries_corrigido':'sum'})

In [63]:
entries_day.head(400)

,station,linename,date,n_entries_corrigido
0,1 AVE,L,2011-01-01,0.0
1,1 AVE,L,2011-01-02,12602.0
2,1 AVE,L,2011-01-03,16041.0
3,1 AVE,L,2011-01-04,21306.0
4,1 AVE,L,2011-01-05,22407.0
...,...,...,...,...
395,103 ST,1,2011-01-31,11768.0
396,103 ST,1,2011-02-01,13508.0
397,103 ST,1,2011-02-02,13433.0
398,103 ST,1,2011-02-03,13856.0


### Aproach adotado para solução do problema

Dado os dados disponíveis, foi escolhido realizar um modelo para predição do fluxo número de pessoas quem entram por dia para uma determinada estação, escolhida de forma arbitrária.

**Treinamento: Safras 2010 - 2014  
Validação: Safra 2015**